In [1]:
import numpy as np
import scipy as sp
from utility import get_best_connected, invert, score
from data_loader import M,F, benchmark_mapping
n = M.shape[0]

KeyboardInterrupt: 

In [3]:
A = M.todense()
B = F.todense()

In [ ]:
m=200
H_M,M_positions = get_best_connected(M,m)
H_F,F_positions = get_best_connected(F,m)
H_M = H_M.todense()
H_F = H_F.todense()
H_F[np.arange(m),np.arange(m)] = 0
H_M[np.arange(m),np.arange(m)] = 0
np.count_nonzero(H_M)/m**2, np.count_nonzero(H_F)/m**2

(0.34615, 0.1805)

In [108]:
swaps = 100000
G=H_F
H=H_M
print(f"Self score={score(H,H,np.arange(m))}")
print(f"Starting score={score(G,H,H_mapping)}")

s = score(G,H,H_mapping)
test = np.zeros(m,dtype=int)

for i in range(swaps):
    np.copyto(test,H_mapping)

    k=2+rng.poisson(0.5)

    v = rng.choice(m,k,replace=False)
    for j in range(k):
        test[v[j]] = H_mapping[v[(j+1)%k]]

    t = score(G,H,test)
    if t>s:
        s=t
        tmp = H_mapping
        H_mapping = test
        test = tmp

    print(f"{i=}, {s=}",end="\r")

print(f"\n Final score={score(G,H,H_mapping)}")

Self score=31926.0
Starting score=6353.0
i=99999, s=6740.0
 Final score=6740.0


In [3]:
rng = np.random.default_rng()

In [80]:
from algorithms import random_swaps, invert

def find_preferences(G,H,iterations = 100, swaps = 10000, start_mapping=None, verbose=True):

    m = G.shape[0]
    result_matrix = np.zeros((m,m),dtype=int)

    if start_mapping is None:
        unmapped_sources = np.arange(m)
        unmapped_targets = np.arange(m)
        start_mapping = np.arange(m)
    else:
        unmapped_sources = np.flatnonzero(start_mapping==-1)
        unmapped_targets = np.ones(m,dtype=bool)
        unmapped_targets[start_mapping[start_mapping!=-1]] = False
        unmapped_targets = np.flatnonzero(unmapped_targets)

    for k in range(iterations):

        mapping = start_mapping.copy()
        mapping[unmapped_sources] = rng.permutation(unmapped_targets)
        random_swaps(G,H,mapping,swaps,verbose=False)
        np.add.at(result_matrix,(np.arange(m),mapping),1)
        if verbose:
            print(f"{k=}"+" " * 20,end="\r")

    return result_matrix

def generate_hint(G,H,iterations,swaps, start_mapping=None):

    m = G.shape[0]
    result_matrix = find_preferences(G,H,iterations,swaps,start_mapping)
    best=result_matrix.argmax(axis=1)
    best_scores = result_matrix[np.arange(m),best]

    score_list = best_scores.argsort()[::-1]
    mapping = -np.ones(m,dtype=int)
    inverse = -np.ones(m,dtype=int)

    for u in score_list:
        if inverse[best[u]] == -1:
            mapping[u] = best[u]
            inverse[best[u]] = u

    unmapped_sources = np.flatnonzero(mapping==-1)
    unmapped_targets = np.flatnonzero(inverse==-1)
    mapping[unmapped_sources] = rng.permutation(unmapped_targets)

    return mapping, result_matrix

In [104]:
start_mapping=-np.ones(m,dtype=int)
start_mapping[[71,7]]=[69,67]
mapping, matrix=generate_hint(G,H,100,10000,start_mapping)
random_swaps(G,H,mapping,10000)
score(G,H,mapping),score(G,G,np.arange(m))

(6637.0, 22239.0)